<a href="https://colab.research.google.com/github/lamtranBKHN/MAIT_2020_LabMemberDectector/blob/origin/MAIT_2020_Model_Training_VGG16_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten

IMG_SIZE = 128

In [2]:

data_dir = '/content/drive/MyDrive/Sample_data_31.12_augmentated'
print(os.listdir(data_dir))
labelNum = (len(os.listdir(data_dir)))

['THHoang_181888', 'THGiang_192200', 'Thanh_Quang', 'TDPhu_181692', 'Phuong_Anh', 'PGPhong_181691', 'NVMinh_174064', 'NVLinh_181579', 'NTTung_181828', 'NTHLy_181637', 'NTHai_173821', 'NTCong_179626', 'NQThai_192002', 'NQAnh_181325', 'NNHoang_181492', 'Nguyen_Thi_Huong', 'Nguyen_Ba_Hoang', 'NDToan_174273', 'NDQuan_192034', 'NDHuy_161823', 'NBHoang_181486', 'LTHien_173846', 'LDTAnh_181320', 'KTAnh_176917', 'HDTuan_192259', 'DPDuong_181432', 'DLTMy_200426', 'DADung_181421', 'CQDat_181383', 'Bui_Quang_Duy', 'VVHung_192210', 'VCThinh_174239', 'TVThai_181749', 'TVSon_181920', 'Tran_Thi_Thanh_Nhan', 'Tran_Minh_Thuyet', 'TQMinh_181659', 'Toan']


In [3]:

# num_skipped = 0

# for fname in os.listdir(data_dir):
#   print("Processing " + fname + " ... ")
#   personal_img_path = os.path.join(data_dir, fname) 
#   for fname in os.listdir(personal_img_path):
#     fpath = os.path.join(personal_img_path, fname)
#     try:
#         fobj = open(fpath, "rb")
#         is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
#     finally:
#         fobj.close()

#     if not is_jfif:
#         num_skipped += 1
#         # Delete corrupted image
#         os.remove(fpath)

# print("Deleted %d images" % num_skipped)

In [4]:
image_size = (IMG_SIZE, IMG_SIZE)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 4356 files belonging to 38 classes.
Using 3485 files for training.
Found 4356 files belonging to 38 classes.
Using 871 files for validation.


In [5]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomContrast(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1, 0.1),
    ]
)

In [ ]:
#from keras.utils import to_categorical
#y = to_categorical(y)
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected lay
baseModel = VGG16(weights='imagenet', include_top=False, \
                  input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
# Buil layer
fcHead = baseModel.output
# Flatten 
fcHead = Flatten()(fcHead)
# Add FC
fcHead = Dense(1000, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)
# Output layer with softmax activation
fcHead = Dense(labelNum, activation='softmax')(fcHead)
# modle
model = model = Model(inputs=baseModel.input, outputs=fcHead)
keras.utils.plot_model(model, show_shapes=True)

In [9]:
epochs = 100

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-3),
#     loss="binary_crossentropy",
#     metrics=["accuracy"],
# )
model.compile(loss = "sparse_categorical_crossentropy", 
                    optimizer = SGD(lr=1e-5, momentum=0.9), 
                    metrics=["accuracy"])
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/100
109/109 [==============================] - 26s 232ms/step - loss: 3.5652 - accuracy: 0.0641 - val_loss: 3.5005 - val_accuracy: 0.1228
Epoch 2/100
109/109 [==============================] - 26s 238ms/step - loss: 3.5369 - accuracy: 0.0691 - val_loss: 3.4457 - val_accuracy: 0.1447
Epoch 3/100
109/109 [==============================] - 26s 238ms/step - loss: 3.4899 - accuracy: 0.0860 - val_loss: 3.3681 - val_accuracy: 0.1366
Epoch 4/100
109/109 [==============================] - 27s 240ms/step - loss: 3.4464 - accuracy: 0.0984 - val_loss: 3.3213 - val_accuracy: 0.1619
Epoch 5/100
109/109 [==============================] - 27s 242ms/step - loss: 3.3777 - accuracy: 0.1126 - val_loss: 3.2391 - val_accuracy: 0.1630
Epoch 6/100
109/109 [==============================] - 27s 242ms/step - loss: 3.2856 - accuracy: 0.1329 - val_loss: 3.1500 - val_accuracy: 0.1699
Epoch 7/100
109/109 [==============================] - 27s 244ms/step - loss: 3.2700 - accuracy: 0.1367 - val_loss: 3.1017 -

In [12]:
img = keras.preprocessing.image.load_img(
    "duy.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

for x in range(len(predictions)): 
    print(predictions[x])


[9.99995828e-01 3.75600395e-09 3.22312281e-11 2.53563348e-09
 1.60574332e-11 7.40789707e-10 8.17396986e-13 9.47865786e-10
 3.34070864e-06 1.27508166e-13 2.23523075e-10 3.59869858e-08
 1.64973868e-09 3.79614265e-12 4.58870443e-14 2.76138418e-10
 5.26000299e-10 3.52080322e-12 1.69302777e-13 3.36339379e-09
 1.57396940e-09 1.36297935e-11 2.49630438e-12 3.96592164e-08
 6.44342761e-13 3.26657590e-11 1.45059978e-10 1.52035251e-09
 3.24564041e-07 1.05275143e-10 1.57854372e-11 1.93023396e-07
 8.28837265e-09 1.63759122e-08 2.82837078e-08 1.59930169e-09
 2.79012852e-07 6.99226788e-11]


In [13]:
img = keras.preprocessing.image.load_img(
    "my.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

for x in range(len(predictions)): 
    print(predictions[x])

[3.94196121e-07 1.31380906e-08 3.47067952e-09 9.99790967e-01
 2.08143769e-09 6.28808960e-09 1.07594954e-10 3.88774986e-08
 2.21713231e-06 7.15934881e-11 1.48090290e-10 3.66814277e-07
 1.92268135e-09 8.31461833e-09 1.37396626e-12 6.12517624e-07
 2.98648530e-07 1.55606347e-06 3.59706043e-09 2.10847002e-08
 5.89490483e-08 1.49903556e-08 1.42613653e-05 1.61342599e-04
 2.44432061e-11 7.64846845e-06 4.64932803e-08 1.18769364e-07
 4.93418746e-08 9.00454597e-06 5.58577398e-08 2.37166931e-09
 6.43163887e-08 1.15691523e-06 7.61120987e-07 8.04589945e-06
 3.52692853e-09 7.25653308e-07]


In [14]:
img = keras.preprocessing.image.load_img(
    "duong.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

for x in range(len(predictions)): 
    print(predictions[x])

[3.2886049e-16 3.1848836e-17 3.3870686e-19 1.7644279e-20 1.0000000e+00
 3.9387664e-17 1.4947989e-17 7.6496426e-14 5.4376563e-22 1.1401014e-18
 6.3367726e-17 5.5095596e-17 2.0834491e-21 1.7887017e-14 2.0830869e-18
 4.9845407e-17 9.6832034e-16 2.5925832e-17 5.7913898e-19 9.6150250e-18
 9.2336956e-13 3.4246776e-21 4.8062151e-19 9.7578702e-19 8.2659080e-17
 1.3608729e-19 2.1780165e-17 3.8413826e-16 1.0152059e-14 2.3397133e-16
 4.8559545e-19 3.9378821e-16 6.2881105e-16 3.1543537e-17 7.9109138e-18
 1.5756576e-17 2.3517441e-20 2.1708166e-17]


In [15]:
img = keras.preprocessing.image.load_img(
    "hung.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

for x in range(len(predictions)): 
    print(predictions[x])

[2.0893437e-12 2.8779168e-09 3.0372849e-10 1.0732280e-07 1.4874214e-09
 1.1473980e-11 1.6640720e-11 4.6098672e-09 8.0306535e-13 4.6282863e-14
 3.2296805e-09 1.2243739e-09 6.5302733e-12 2.4792683e-09 9.5901184e-13
 1.5137025e-10 1.3036382e-12 2.0621287e-12 8.0053061e-11 2.5111898e-08
 3.3264516e-07 1.3907800e-10 1.5093672e-09 6.5660821e-08 8.2056757e-14
 4.5481834e-06 3.1415047e-07 9.1121636e-14 1.0843174e-09 5.9670505e-11
 7.3954724e-14 4.2462472e-10 2.1371509e-11 4.0094628e-13 7.1702397e-07
 8.9623371e-11 2.0362732e-13 9.9999392e-01]


In [16]:
img = keras.preprocessing.image.load_img(
    "thinh.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

for x in range(len(predictions)): 
    print(predictions[x])

[3.91075085e-13 2.62005976e-14 2.70685493e-15 2.64765326e-15
 2.90379244e-12 1.20702784e-18 3.28532544e-18 1.59887260e-14
 2.06723981e-15 1.07668380e-15 7.17475531e-15 9.51418323e-15
 2.29786528e-15 1.02925836e-14 2.84859213e-14 2.72914633e-08
 9.64968994e-09 4.82567503e-15 7.34379881e-17 1.15844053e-11
 1.38212085e-14 1.13534402e-17 1.44112492e-15 6.01378517e-16
 2.86888128e-14 1.47707326e-16 1.47472722e-14 1.35403742e-17
 3.25874771e-13 1.16136690e-12 3.88541629e-11 3.23280985e-12
 1.08304482e-06 5.79893663e-14 4.11655065e-14 6.50019542e-16
 9.99998927e-01 1.14100136e-17]


In [17]:
img = keras.preprocessing.image.load_img(
    "thinh1.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

for x in range(len(predictions)): 
    print(predictions[x])

[1.27084480e-04 5.90847503e-06 4.49258465e-07 7.18874105e-07
 6.60969019e-01 1.24570164e-07 1.49691425e-06 2.91577635e-07
 4.78482187e-09 5.74827391e-05 1.02049971e-05 2.23889565e-05
 6.66357846e-06 1.96812898e-01 7.44584613e-05 9.58579039e-05
 5.51649828e-06 1.52599730e-03 1.37407341e-07 4.05064435e-04
 7.40416654e-05 3.02085624e-07 1.23809878e-04 1.30097376e-07
 9.74517825e-05 7.94632893e-09 9.67528990e-07 5.94190788e-05
 1.30304411e-01 3.12189659e-04 5.02998319e-07 1.05449915e-04
 6.35058060e-03 9.64993960e-05 2.28970335e-03 4.49321487e-05
 1.64587309e-05 1.34444213e-06]
